In [ ]:
#|default_exp lookup
#|default_cls_lvl 3

# Doc lookup

> Convert backticks to links

In [ ]:
#|export
from nbprocess.imports import *
from nbprocess.read import *
from nbprocess.export import *
from nbprocess.doclinks import *
from fastcore.utils import *

import pkg_resources,importlib

if IN_NOTEBOOK:
    from IPython.display import Markdown,display
    from IPython.core import page
else: Markdown,display,page = None,None,None

In [ ]:
#|hide
from fastcore.test import *

In [17]:
#|export
class NbdevLookup:
    "Mapping from symbol names to URLs with docs"
    def __init__(self, strip_libs=None, incl_libs=None, skip_mods=None):
        skip_mods = setify(skip_mods)
        strip_libs = L(strip_libs)
        if incl_libs is not None: incl_libs = (L(incl_libs)+strip_libs).unique()
        # Dict from lib name to _nbprocess module for incl_libs (defaults to all)
        self.entries = {o.name: o.load() for o in pkg_resources.iter_entry_points(group='nbdev')
                       if incl_libs is None or o.dist.key in incl_libs}
        py_syms = merge(*L(o['syms'].values() for o in self.entries.values()).concat())
        for m in strip_libs:
            _d = self.entries[m]
            stripped = {remove_prefix(k,f"{mod}."):v
                        for mod,dets in _d['syms'].items() if mod not in skip_mods
                        for k,v in dets.items()}
            py_syms = merge(stripped, py_syms)
        self.syms = py_syms

    def __getitem__(self, s): return self.syms.get(s, None)

Symbol names are taken from libraries registered using the 'nbprocess' entry point. By default, all libraries with this entry point are searched, but full symbol names (including module prefix) are required.

In [19]:
c = NbdevLookup()
assert c['nbprocess.doclinks.DocLinks'].startswith('http')
assert c['numpy.array'].startswith('http')
assert not c['DocLinks']

TypeError: 'module' object is not subscriptable

Pass `strip_libs` to list libraries which should be available without requiring a module prefix.

In [16]:
c = NbdevLookup(strip_libs=['nbprocess'])
assert c['array'].startswith('http')
assert c['DocLinks'].startswith('http')

AttributeError: 'NoneType' object has no attribute 'startswith'

In [ ]:
#|export
from nbprocess._modidx import d as modidx
_settings = modidx['settings']
_strip_libs  = _settings.get('strip_libs',_settings.get('lib_name')).split()
nbprocess_lookup = NbdevLookup(_strip_libs)

nbprocess itself includes `nbdev_lookup`, an instantiated `NbdevLookup` with `strip_libs=nbprocess`.

In [ ]:
assert nbprocess_lookup['DocLinks'].startswith('http')
assert nbprocess_lookup['numpy.array'].startswith('http')
assert not nbprocess_lookup['array']

## Backticks

In [ ]:
#|export
@patch
def _link_sym(self:NbdevLookup, m):
    l = m.group(1)
    s = self[l]
    if s is None: return m.group(0)
    return rf"[{l}]({s})"

_re_backticks = re.compile(r'`([^`\s]+)`')
@patch
def _link_line(self:NbdevLookup, l): return _re_backticks.sub(self._link_sym, l)

@patch
def linkify(self:NbdevLookup, md):
    if md:
        in_fence=False
        lines = md.splitlines()
        for i,l in enumerate(lines):
            if l.startswith("```"): in_fence=not in_fence
            elif not l.startswith('    ') and not in_fence: lines[i] = self._link_line(l)
        return '\n'.join(lines)

In [13]:
md = """This is a link to `numpy.array` and to `read_nb` but not a link to `foobar`.
And not a link to <code>dict2nb</code>.

    This is not a link to `read_nb`

```
This isn't a link to `read_nb` either
```"""

In [14]:
#|eval: false
c = NbdevLookup('nbprocess')
Markdown(c.linkify(md))

This is a link to `numpy.array` and to [read_nb](https://nbprocess.fast.ai/nbprocess.read#read_nb) but not a link to `foobar`.
And not a link to <code>dict2nb</code>.

    This is not a link to `read_nb`

```
This isn't a link to `read_nb` either
```

## Export -

In [ ]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()